In [1]:
# import libraries
#try:
  # %tensorflow_version only exists in Colab.
#  !pip install tf-nightly
#except Exception:
#  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
#!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.11.0


In [2]:
# get data files

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [3]:
train_data=pd.read_csv(train_file_path, delimiter="\t", header=None)
test_data=pd.read_csv(test_file_path, delimiter="\t", header=None)
train_data.head()

,0,1
0,ham,ahhhh...just woken up!had a bad dream about u ...
1,ham,you can never do nothing
2,ham,"now u sound like manky scouse boy steve,like! ..."
3,ham,mum say we wan to go then go... then she can s...
4,ham,never y lei... i v lazy... got wat? dat day ü ...


In [35]:
labels_train=[1 if i=="ham" else 0 for i in train_data[0]]
labels_test=[1 if i=="ham" else 0 for i in test_data[0]]

set_vocab=set()
for i in train_data[1]:
    set_vocab.update(set(i.split()))
print(len(set_vocab))

11330


In [37]:
import string
def encode_and_pad(sms_messages, set_vocab, max_length=200):
    dict_encodings={}
    count=1
    for i in set_vocab:
        #print(str.maketrans('', '', string.punctuation))
        dict_encodings[i.translate(str.maketrans('', '', string.punctuation)).lower()]=count
        count=count+1
    encoded_sms=[[dict_encodings[j.translate(str.maketrans('', '', string.punctuation)).lower()] for j in i.split(" ")] for i in train_data[1]]

    for i in range(len(encoded_sms)):
        if len(encoded_sms[i])< max_length:
            #encoded_sms[i]=np.asarray(encoded_sms[i] + ((max_length-len(encoded_sms[i]))*[0])).astype("float32")
            encoded_sms[i]=((max_length-len(encoded_sms[i]))*[0]) + encoded_sms[i] 
    return np.array(encoded_sms)

encoded_train_sms=encode_and_pad(train_data[1], set_vocab)    
encoded_test_sms=encode_and_pad(test_data[1], set_vocab)    
#print(np.array(encoded_train_sms))


In [60]:
model = keras.Sequential()

model.add(keras.layers.Embedding(
    11331,
    100,
    input_length=200)),
model.add(keras.layers.Flatten())
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(100, activation='sigmoid'))
model.add(keras.layers.Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#monitor = keras.callbacks.EarlyStopping(monitor='val_acc', min_delta=1e-4, patience=25, verbose=1, mode='max', restore_best_weights=True)

labels_train=np.array(labels_train).astype('float32').reshape((-1,1))
labels_test=np.array(labels_test).astype('float32').reshape((-1,1))
#labels_train
encoded_train_sms

array([[    0,     0,     0, ..., 10436,  9761, 11165],
       [    0,     0,     0, ...,  9292, 10853,  7673],
       [    0,     0,     0, ...,  2841, 10534,  7659],
       ...,
       [    0,     0,     0, ...,  8145, 10692,  8020],
       [    0,     0,     0, ...,  9574,  4130, 10824],
       [    0,     0,     0, ...,  1486,  2430,  9543]])

In [61]:
#model.fit(encoded_train_sms, labels_train, callbacks=[monitor], epochs=10)
model.fit(encoded_train_sms, labels_train, epochs=20)

Epoch 1/10
131/131 [==============================] - 9s 52ms/step - loss: 0.1907 - accuracy: 0.9301
Epoch 2/10
131/131 [==============================] - 6s 49ms/step - loss: 0.0396 - accuracy: 0.9899
Epoch 3/10
131/131 [==============================] - 6s 48ms/step - loss: 0.0168 - accuracy: 0.9959
Epoch 4/10
131/131 [==============================] - 7s 51ms/step - loss: 0.0068 - accuracy: 0.9990
Epoch 5/10
131/131 [==============================] - 6s 47ms/step - loss: 0.0035 - accuracy: 0.9995
Epoch 6/10
131/131 [==============================] - 6s 49ms/step - loss: 0.0023 - accuracy: 0.9998
Epoch 7/10
131/131 [==============================] - 6s 49ms/step - loss: 0.0013 - accuracy: 0.9998
Epoch 8/10
131/131 [==============================] - 6s 49ms/step - loss: 8.3435e-04 - accuracy: 1.0000
Epoch 9/10
131/131 [==============================] - 7s 52ms/step - loss: 6.5421e-04 - accuracy: 1.0000
Epoch 10/10
131/131 [==============================] - 6s 48ms/step - loss: 4.6374e

In [90]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text, set_vocab=set_vocab):
    dict_encodings={}
    count=1
    for i in set_vocab:
        #print(str.maketrans('', '', string.punctuation))
        dict_encodings[i.translate(str.maketrans('', '', string.punctuation)).lower()]=count
        count=count+1
    encoded_msg=[]
    for i in pred_text.split():
        formatted_string=i.translate(str.maketrans('', '', string.punctuation)).lower()
        if formatted_string not in dict_encodings:
            encoded_msg.append(0)
        else:
            encoded_msg.append(dict_encodings[formatted_string])
    encoded_msg=((200-len(encoded_msg))*[0])+encoded_msg
    encoded_msg=np.array(encoded_msg).reshape((1,-1))
    prediction1=model.predict(encoded_msg).item()
    prediction2="ham" if prediction1>0.5 else "spam"
    prediction=[prediction1, prediction2]

    return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 [==============================] - 0s 65ms/step
[0.9997586011886597, 'ham']


In [91]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 [==============================] - 0s 50ms/step
You passed the challenge. Great job!
